# Chatbot

Les chatbots sont extrêmement utiles pour les entreprises et les clients. La majorité des gens préfèrent parler directement à partir d'une boîte de dialogue au lieu d'appeler les centres de service. Facebook a publié des données qui ont prouvé la valeur des bots. Plus de 2 milliards de messages sont envoyés chaque mois entre les particuliers et les entreprises. L'étude HubSpot nous indique que 71% des personnes souhaitent bénéficier du support client des applications de messagerie. C'est un moyen rapide de résoudre leurs problèmes afin que les chatbots aient un bel avenir dans les organisations.

Aujourd'hui, nous allons construire un  chatbot à partir de zéro qui sera capable de comprendre de quoi parle l'utilisateur et de donner une réponse appropriée.

## Comment fonctionnent les chatbots?

Les chatbots ne sont rien d'autre qu'un logiciel intelligent qui peut interagir et communiquer avec des personnes comme les humains.

Tous les chatbots relèvent des concepts de NLP (Natural Language Processing). La NLP est composée de deux choses:

*     **NLU** (Natural Language Understanding): La capacité des machines à comprendre le langage humain comme l'anglais.

*     **NLG** (Natural Language Generation): capacité d'une machine à générer du texte similaire à des phrases écrites par des humains.

Imaginez un utilisateur posant une question à un chatbot: "Hé, qu'est-ce qu'il y a dans l'actualité aujourd'hui?"

Le chatbot décomposera la phrase de l'utilisateur en deux choses: l'intention et une entité. L'intention de cette phrase peut être get_news car elle fait référence à une action que l'utilisateur souhaite effectuer. L'entité donne des détails spécifiques sur l'intention, donc "aujourd'hui" sera l'entité. Ainsi, un modèle d'apprentissage automatique est utilisé pour reconnaître les intentions et les entités du chat.

## Code

### Step 1. Importez les bibliothèques et chargez les données


In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

D:\Conda\Anaconda\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
D:\Conda\Anaconda\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/rel

In [2]:
intents_file = open('data/Liste_d_intention.json', encoding='utf-8').read()
intents = json.loads(intents_file)

In [3]:
listInt = intents['intentions']

In [4]:
for i in  range(len(listInt)):
    print(i,listInt[i]['tag'] )
    listInt[i]['questions']

0 salutations
1 piecesAfournir
2 domaine 
3 specialisation
4 diplomesDelivres
5 modalite
6 biblioteque
7 localisation
8 conditionsDAdmissibilites
9 debutDesCours
10 remerciements
11 auRevoir
12 


### Step 2. Prétraitement des données

Le modèle ne peut pas prendre les données brutes. Il doit passer par beaucoup de prétraitement pour que la machine le comprenne facilement. Pour les données textuelles, il existe de nombreuses techniques de prétraitement disponibles. La première technique est la tokenisation, dans laquelle nous divisons les phrases en mots.

En observant le fichier d'intents, nous pouvons voir que chaque balise contient une liste de modèles et de réponses. Nous tokenisons chaque modèle et ajoutons les mots dans une liste. De plus, nous créons une liste de classes et de documents pour ajouter toutes les intentions associées aux modèles.

In [5]:
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

for intent in intents['intentions']:
    for pattern in intent['questions']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)        
        documents.append((word, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

words[:5]

[(['Bonjour'], 'salutations'), (['Comment', 'allez-vous', '?'], 'salutations'), (['Salut'], 'salutations'), (['Quoi', 'de', 'neuf'], 'salutations'), (['Salut', '!'], 'salutations'), (['Ça', 'va', '?'], 'salutations'), (['quelle', 'sont', 'les', 'différentes', 'pieces', 'demander', '?'], 'piecesAfournir'), (['quelle', 'sont', 'les', 'papiers', 'que', 'vous', 'demendez', '?'], 'piecesAfournir'), (['dites', 'moi', 'les', 'pieces', 'à', 'amener', 'pour', "s'inscrir", '?'], 'piecesAfournir'), (['donner', 'moi', 'les', 'papiers', 'demender', 'lors', 'de', "l'inscription", '?'], 'piecesAfournir'), (['citez', 'moi', 'les', 'pieces', 'dont', 'on', 'a', 'besoin', 'pour', "s'inscrir", '?'], 'piecesAfournir'), (['pouriez', 'vous', 'me', 'donner', 'des', 'renseignement', 'sur', 'les', 'papiers', 'à', 'fournir', '?'], 'piecesAfournir'), (['votre', 'formation', 'est', 'accée', 'sur', 'quel', 'domaine', '?'], 'domaine '), (['quels', 'sont', 'les', 'domaines', 'sur', 'lesquels', 'vous', 'dispensez', 'v

['Bonjour', 'Comment', 'allez-vous', '?', 'Salut']

Une autre technique est la lemmatisation. Nous pouvons convertir les mots en forme de lemme afin de réduire tous les mots canoniques. Par exemple, les mots play, playing, played, players, etc. seront tous remplacés par play. De cette façon, nous pouvons réduire le nombre total de mots dans notre vocabulaire. Alors maintenant, nous lemmatisons chaque mot et supprimons les mots en double.

In [6]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('checkpoints/words.pkl','wb'))
pickle.dump(classes,open('checkpoints/classes.pkl','wb'))


55 documents
12 classes ['auRevoir', 'biblioteque', 'conditionsDAdmissibilites', 'debutDesCours', 'diplomesDelivres', 'domaine ', 'localisation', 'modalite', 'piecesAfournir', 'remerciements', 'salutations', 'specialisation']
136 unique lemmatized words ["'", 'a', 'accée', 'adresse', 'allez-vous', 'amener', 'au', 'auquels', 'avez', 'avez-vous', 'avoir', 'besoin', 'biblioteque', 'bonjour', "c'est", 'ce', 'citez', 'combien', 'comment', 'condition', 'conditios', 'cours', 'cout', "d'admission", "d'eclairecissement", 'dans', 'de', 'debut', 'debuter', 'demander', 'demender', 'demendez', 'differente', 'differentes', 'différentes', 'diplomes', 'diplôme', 'dispensez', 'dites', 'domaine', 'domaines', 'donner', 'dont', 'délivrance', 'délivrez', 'délivré', 'démarage', 'en', 'est', 'est-ce', 'faire', 'faite', 'formation', 'forme', 'fournir', 'genre', 'geographique', 'information', 'ipd', 'je', "l'ecole", "l'inscription", "l'ipd", 'la', 'le', 'lesquels', 'localisation', 'lors', "m'informer", 'me', '

A la fin, **words** contiennent le vocabulaire de notre projet et **classes** contiennent le total des entités à classer. Pour enregistrer l'objet python dans un fichier, nous avons utilisé la méthode pickle.dump (). Ces fichiers seront utiles une fois la formation terminée et nous prédisons les discussions.

### Step 3. Créer des données d'entrainement et de test

Pour entraîner le modèle, nous convertirons chaque modèle d'entrée en nombres. Tout d'abord, nous allons lemmatiser chaque mot du motif et créer une liste de zéros de même longueur que le nombre total de mots. Nous définirons la valeur 1 uniquement sur les index contenant le mot dans les modèles. De la même manière, nous allons créer la sortie en définissant 1 sur l'entrée de classe à laquelle appartient le motif.

In [7]:
# create the training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data is created")

Training data is created


D:\Conda\Anaconda\lib\site-packages\ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  from ipykernel import kernelapp as app


### Step 4. Formation du modèle

L'architecture de notre modèle sera un réseau de neurones composé de 3 couches denses. La première couche a 128 neurones, la seconde en a 64 et la dernière couche aura les mêmes neurones que le nombre de classes. Les couches d'abandon sont introduites pour réduire le surajustement du modèle. Nous avons utilisé l'optimiseur SGD et ajusté les données pour démarrer l'apprentissage du modèle. Une fois l'apprentissage de 200 époques terminé, nous sauvegardons le modèle entraîné à l'aide de la fonction Keras model.save («chatbot_model.h5»).

In [8]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=900, batch_size=8)
model.save('checkpoints/chatbot_model.h5', hist)
print("model is created")

D:\Conda\Anaconda\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/900
7/7 [==============================] - 0s 2ms/step - loss: 2.4757 - accuracy: 0.0909
Epoch 2/900
7/7 [==============================] - 0s 2ms/step - loss: 2.4559 - accuracy: 0.0727
Epoch 3/900
7/7 [==============================] - 0s 2ms/step - loss: 2.3757 - accuracy: 0.1636
Epoch 4/900
7/7 [==============================] - 0s 2ms/step - loss: 2.2625 - accuracy: 0.1636
Epoch 5/900
7/7 [==============================] - 0s 2ms/step - loss: 2.1943 - accuracy: 0.2545
Epoch 6/900
7/7 [==============================] - 0s 2ms/step - loss: 2.1325 - accuracy: 0.2545
Epoch 7/900
7/7 [==============================] - 0s 2ms/step - loss: 2.1198 - accuracy: 0.2909
Epoch 8/900
7/7 [==============================] - 0s 2ms/step - loss: 2.0079 - accuracy: 0.3818
Epoch 9/900
7/7 [==============================] - 0s 2ms/step - loss: 1.8965 - accuracy: 0.3273
Epoch 10/900
7/7 [==============================] - 0s 2ms/step - loss: 1.8894 - accuracy: 0.4182
Epoch 11/900
7/7 [===========

7/7 [==============================] - 0s 1ms/step - loss: 0.0733 - accuracy: 1.0000
Epoch 85/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0923 - accuracy: 0.9818
Epoch 86/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0931 - accuracy: 0.9818
Epoch 87/900
7/7 [==============================] - 0s 1ms/step - loss: 0.1469 - accuracy: 0.9455
Epoch 88/900
7/7 [==============================] - 0s 2ms/step - loss: 0.1287 - accuracy: 0.9818
Epoch 89/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0847 - accuracy: 1.0000
Epoch 90/900
7/7 [==============================] - 0s 1ms/step - loss: 0.1935 - accuracy: 0.9455
Epoch 91/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0990 - accuracy: 0.9818
Epoch 92/900
7/7 [==============================] - 0s 2ms/step - loss: 0.1291 - accuracy: 0.9636
Epoch 93/900
7/7 [==============================] - 0s 2ms/step - loss: 0.1397 - accuracy: 0.9636
Epoch 94/900
7/7 [===============

7/7 [==============================] - 0s 2ms/step - loss: 0.0837 - accuracy: 0.9636
Epoch 168/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0385 - accuracy: 1.0000
Epoch 169/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0613 - accuracy: 0.9818
Epoch 170/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0150 - accuracy: 1.0000
Epoch 171/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0608 - accuracy: 1.0000
Epoch 172/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0586 - accuracy: 1.0000
Epoch 173/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0651 - accuracy: 0.9818
Epoch 174/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0456 - accuracy: 0.9818
Epoch 175/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0532 - accuracy: 1.0000
Epoch 176/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0210 - accuracy: 1.0000
Epoch 177/900
7/7 [=====

7/7 [==============================] - 0s 1ms/step - loss: 0.0502 - accuracy: 0.9636
Epoch 250/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0665 - accuracy: 0.9636
Epoch 251/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0340 - accuracy: 0.9818
Epoch 252/900
7/7 [==============================] - 0s 1ms/step - loss: 0.1396 - accuracy: 0.9636
Epoch 253/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0234 - accuracy: 1.0000
Epoch 254/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0545 - accuracy: 0.9818
Epoch 255/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0140 - accuracy: 1.0000
Epoch 256/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0323 - accuracy: 0.9818
Epoch 257/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0045 - accuracy: 1.0000
Epoch 258/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0220 - accuracy: 1.0000
Epoch 259/900
7/7 [=====

...

7/7 [==============================] - 0s 2ms/step - loss: 0.0176 - accuracy: 1.0000
Epoch 496/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0429 - accuracy: 0.9818
Epoch 497/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0174 - accuracy: 1.0000
Epoch 498/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0254 - accuracy: 0.9818
Epoch 499/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0187 - accuracy: 0.9818
Epoch 500/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0553 - accuracy: 0.9636
Epoch 501/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 502/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0376 - accuracy: 0.9818
Epoch 503/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0133 - accuracy: 1.0000
Epoch 504/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0269 - accuracy: 1.0000
Epoch 505/900
7/7 [=====

7/7 [==============================] - 0s 1ms/step - loss: 0.0141 - accuracy: 1.0000
Epoch 578/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0189 - accuracy: 0.9818
Epoch 579/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0067 - accuracy: 1.0000
Epoch 580/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 581/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0053 - accuracy: 1.0000
Epoch 582/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0201 - accuracy: 0.9818
Epoch 583/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0121 - accuracy: 1.0000
Epoch 584/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 585/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0485 - accuracy: 0.9818
Epoch 586/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 587/900
7/7 [=====

7/7 [==============================] - 0s 2ms/step - loss: 0.0165 - accuracy: 1.0000
Epoch 660/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0075 - accuracy: 1.0000
Epoch 661/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0341 - accuracy: 0.9818
Epoch 662/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 663/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 664/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0125 - accuracy: 1.0000
Epoch 665/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 666/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0255 - accuracy: 1.0000
Epoch 667/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0062 - accuracy: 1.0000
Epoch 668/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0159 - accuracy: 0.9818
Epoch 669/900
7/7 [=====

7/7 [==============================] - 0s 1ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 742/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0099 - accuracy: 1.0000
Epoch 743/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 744/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0148 - accuracy: 1.0000
Epoch 745/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 746/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 747/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 748/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 749/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0097 - accuracy: 1.0000
Epoch 750/900
7/7 [==============================] - 0s 1ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 751/900
7/7 [=====

7/7 [==============================] - 0s 3ms/step - loss: 0.0103 - accuracy: 1.0000
Epoch 824/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0276 - accuracy: 0.9818
Epoch 825/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 826/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0053 - accuracy: 1.0000
Epoch 827/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0208 - accuracy: 1.0000
Epoch 828/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0284 - accuracy: 1.0000
Epoch 829/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0052 - accuracy: 1.0000
Epoch 830/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 831/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 832/900
7/7 [==============================] - 0s 2ms/step - loss: 0.0272 - accuracy: 0.9818
Epoch 833/900
7/7 [=====

### Step 5. Interagir avec le chatbot

Notre modèle est prêt à chatter, alors créons maintenant une belle interface utilisateur graphique pour notre chatbot dans un nouveau fichier. Vous pouvez nommer le fichier comme gui_chatbot.py

Dans notre fichier GUI, nous utiliserons le module Tkinter pour construire la structure de l'application de bureau, puis nous capturerons le message de l'utilisateur et effectuerons à nouveau un prétraitement avant d'entrer le message dans notre modèle entraîné.

Le modèle prédira ensuite l'étiquette du message de l'utilisateur et nous sélectionnerons au hasard la réponse dans la liste des réponses de notre fichier d'intentions.

Voici le code source complet du fichier GUI.

In [9]:
import nltk
import pickle
import numpy as np
from nltk.stem import WordNetLemmatizer
from keras.models import load_model
import json
import random


lemmatizer = WordNetLemmatizer()
model = load_model('checkpoints/chatbot_model.h5')

intents = json.loads(open('data/Liste_d_intention.json', encoding='utf-8').read())
words = pickle.load(open('checkpoints/words.pkl','rb'))
classes = pickle.load(open('checkpoints/classes.pkl','rb'))

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

    
def bag_of_words(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                 bag[i] = 1
                 if show_details:
                     print ("found in bag: %s" % word)
    return(np.array(bag))


def predict_class(sentence):
     p = bag_of_words(sentence, words,show_details=False)
     res = model.predict(np.array([p]))[0]
     ERROR_THRESHOLD = 0.25
     results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

     results.sort(key=lambda x: x[1], reverse=True)
     return_list = []
     for r in results:
         return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
     return return_list

def getResponse(ints, intents_json):
     tag = ints[0]['intent']
     list_of_intents = intents_json['intentions']
     for i in list_of_intents:
         if(i['tag']== tag):
             result = random.choice(i['réponses'])
             break
     return result

In [ ]:


#Creating tkinter GUI
import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatBox.config(state=NORMAL)
        ChatBox.insert(END, "You: " + msg + '\n\n')
        ChatBox.config(foreground="#446665", font=("Verdana", 12 )) 
        ints = predict_class(msg)
        res = getResponse(ints, intents)
        ChatBox.insert(END, "Bot: " + res + '\n\n')           
        ChatBox.config(state=DISABLED)
        ChatBox.yview(END)

root = Tk()
root.title("Chatbot")
root.geometry("400x500")
root.resizable(width=False, height=False)

#Create Chat window
ChatBox = Text(root, bd=0, bg="white", height="20", width="50", font="Arial",)
ChatBox.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
ChatBox['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(root, font=("Verdana",12,'bold'), 
                    text="Send", width="10", height="4",
                    bd=0, bg="#f9a602", activebackground="#3c9d9b",
                    fg='#000000',command= send )


#Create the box to enter message
EntryBox = Text(root, bd=0, bg="white",width="40", height="4", font="Arial")

#EntryBox.bind("<Return>", send)
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatBox.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
root.mainloop()
